<a href="https://colab.research.google.com/github/username/repo/blob/main/Google_Manus_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Google Manus System

A powerful AI assistant system that leverages Google's local models in Colab for code generation, execution, research, and web deployment.

## Features

- 🧠 Uses Google's local models (Gemini, Gemma)
- 💻 Code generation and execution
- 🔍 Research and information retrieval
- 🌐 Web deployment with FRP tunneling
- 🛠️ Extensible tool system

## How to Use

1. Run Box 1 to set up the environment
2. Run Box 2 to initialize the model system
3. Run Box 3 to set up the tool registry
4. Run Box 4 to launch the user interface
5. Run Box 5 (optional) to set up web deployment

Let's get started!

# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🔧 BOX 1: Environment Setup and Configuration - v1.0                                                     ║
# ║                                                                                                          ║
# ╟────────────────────────────────────── CORE FEATURES ─────────────────────────────────────────────────────╢
# ║ - System initialization and dependency installation                                                      ║
# ║ - Configuration management and directory structure setup                                                 ║
# ║ - Environment detection (Colab, local)                                                                   ║
# ║ - Logging and error handling                                                                             ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

In [ ]:
print("🔧 BOX 1: Initializing Environment Setup and Configuration...")

import os
import sys
import json
import time
import logging
import subprocess
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Union, Callable

# Check if running in Colab
try:
    import google.colab
    IS_COLAB = True
    print("✅ Running in Google Colab environment")
except ImportError:
    IS_COLAB = False
    print("ℹ️ Running in local environment")

# Define base directories
if IS_COLAB:
    # Check if Google Drive is mounted
    if not os.path.exists("/content/drive"):
        from google.colab import drive
        drive.mount('/content/drive')
        print("✅ Google Drive mounted")
    
    # Use Google Drive for persistence if available
    BASE_DIR = Path("/content/drive/MyDrive/GoogleManusSystem")
    if not BASE_DIR.exists():
        BASE_DIR.mkdir(parents=True, exist_ok=True)
        print(f"✅ Created base directory at {BASE_DIR}")
    
    # Local working directory for temporary files
    WORKING_DIR = Path("/content/GoogleManusSystem")
else:
    # Local environment setup
    BASE_DIR = Path(os.getcwd()) / "GoogleManusSystem"
    WORKING_DIR = BASE_DIR

# Create necessary subdirectories
WORKSPACE_DIR = BASE_DIR / "workspace"
CONFIG_DIR = BASE_DIR / "config"
LOGS_DIR = BASE_DIR / "logs"
TOOLS_DIR = BASE_DIR / "tools"
SITES_DIR = BASE_DIR / "sites"

for directory in [WORKSPACE_DIR, CONFIG_DIR, LOGS_DIR, TOOLS_DIR, SITES_DIR]:
    directory.mkdir(parents=True, exist_ok=True)
    print(f"✅ Ensured directory exists: {directory}")

# Setup logging
LOG_FILE = LOGS_DIR / "manus_log.json"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("GoogleManus")

# Function to log events in JSON format
def log_event(event_type: str, details: Dict[str, Any]) -> None:
    """Log an event to the JSON log file."""
    event = {
        "timestamp": datetime.now().isoformat(),
        "type": event_type,
        **details
    }
    
    # Append to log file
    with open(LOG_FILE, "a") as f:
        f.write(json.dumps(event) + "\n")
    
    # Also log to console
    logger.info(f"{event_type}: {details}")

# Install required dependencies
required_packages = [
    "fastapi",
    "uvicorn",
    "pydantic",
    "requests",
    "nest_asyncio",
    "ipywidgets",
    "beautifulsoup4",
    "markdown",
    "python-multipart"
]

print("📦 Installing required packages...")
for package in required_packages:
    try:
        __import__(package.split("==")[0].strip())
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"⏳ Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ Installed {package}")

# Apply nest_asyncio for Jupyter compatibility
import nest_asyncio
nest_asyncio.apply()
print("✅ Applied nest_asyncio for Jupyter compatibility")

# Create or load configuration
config_file = CONFIG_DIR / "system_config.json"
if config_file.exists():
    with open(config_file, "r") as f:
        config = json.load(f)
    print("✅ Loaded existing configuration")
else:
    # Default configuration
    config = {
        "version": "1.0.0",
        "created_at": datetime.now().isoformat(),
        "updated_at": datetime.now().isoformat(),
        "default_model": "google/gemini-2.5-pro",
        "workspace_path": str(WORKSPACE_DIR),
        "log_path": str(LOG_FILE),
        "sites_path": str(SITES_DIR),
        "is_colab": IS_COLAB,
        "public_url": None,
        "dashboard_url": None
    }
    
    with open(config_file, "w") as f:
        json.dump(config, f, indent=2)
    print("✅ Created new configuration")

# Export configuration for other boxes
box1_exports = {
    "BASE_DIR": str(BASE_DIR),
    "WORKSPACE_DIR": str(WORKSPACE_DIR),
    "CONFIG_DIR": str(CONFIG_DIR),
    "LOGS_DIR": str(LOGS_DIR),
    "TOOLS_DIR": str(TOOLS_DIR),
    "SITES_DIR": str(SITES_DIR),
    "LOG_FILE": str(LOG_FILE),
    "IS_COLAB": IS_COLAB,
    "config": config
}

box1_exports_file = CONFIG_DIR / "box1_exports.json"
with open(box1_exports_file, "w") as f:
    json.dump(box1_exports, f, indent=2)

print("\n🎉 BOX 1 SETUP COMPLETED SUCCESSFULLY!")
print(f"📁 Base Directory: {BASE_DIR}")
print(f"💾 Configuration: {config_file}")
print(f"📝 Log File: {LOG_FILE}")

# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🧠 BOX 2: Google Model Integration and Core Services - v1.0                                              ║
# ║                                                                                                          ║
# ╟────────────────────────────────────── CORE FEATURES ─────────────────────────────────────────────────────╢
# ║ - Google model integration (Gemini, Gemma)                                                               ║
# ║ - Model selection and configuration                                                                      ║
# ║ - Streaming response handling                                                                            ║
# ║ - Context and memory management                                                                          ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

In [ ]:
print("🧠 BOX 2: Initializing Google Model Integration and Core Services...")

import os
import sys
import json
import time
import asyncio
from pathlib import Path
from typing import Dict, List, Any, Optional, Union, Callable, Generator

# Load configuration from Box 1
try:
    with open(Path("./GoogleManusSystem/config/box1_exports.json"), "r") as f:
        box1_exports = json.load(f)
    
    # Extract paths and configuration
    BASE_DIR = Path(box1_exports["BASE_DIR"])
    CONFIG_DIR = Path(box1_exports["CONFIG_DIR"])
    IS_COLAB = box1_exports["IS_COLAB"]
    config = box1_exports["config"]
    
    print("✅ Loaded configuration from Box 1")
except Exception as e:
    print(f"❌ Error loading Box 1 configuration: {e}")
    print("⚠️ Using default configuration")
    
    # Default configuration if Box 1 hasn't been run
    BASE_DIR = Path("./GoogleManusSystem")
    CONFIG_DIR = BASE_DIR / "config"
    IS_COLAB = True
    config = {"default_model": "google/gemini-2.5-pro"}

# Import Google Colab AI module
try:
    from google.colab import ai
    GOOGLE_AI_AVAILABLE = True
    print("✅ Google Colab AI module imported successfully")
except ImportError:
    GOOGLE_AI_AVAILABLE = False
    print("⚠️ Google Colab AI module not available. Using fallback mode.")

# Get available models
if GOOGLE_AI_AVAILABLE:
    try:
        available_models = ai.list_models()
        print(f"✅ Available models: {available_models}")
    except Exception as e:
        print(f"❌ Error listing models: {e}")
        available_models = [
            "google/gemini-2.0-flash",
            "google/gemini-2.0-flash-lite",
            "google/gemini-2.5-flash",
            "google/gemini-2.5-flash-lite",
            "google/gemini-2.5-pro",
            "google/gemma-3-12b",
            "google/gemma-3-1b",
            "google/gemma-3-27b",
            "google/gemma-3-4b"
        ]
else:
    # Fallback model list
    available_models = [
        "google/gemini-2.0-flash",
        "google/gemini-2.0-flash-lite",
        "google/gemini-2.5-flash",
        "google/gemini-2.5-flash-lite",
        "google/gemini-2.5-pro",
        "google/gemma-3-12b",
        "google/gemma-3-1b",
        "google/gemma-3-27b",
        "google/gemma-3-4b"
    ]

# Set default model
default_model = config.get("default_model", "google/gemini-2.5-pro")
if default_model not in available_models:
    default_model = available_models[0]
    print(f"⚠️ Default model not available. Using {default_model} instead.")
else:
    print(f"✅ Using default model: {default_model}")

# Text generation function
def generate_text(prompt: str, model_name: str = None, stream: bool = False, **kwargs) -> Union[str, Generator]:
    """Generate text using Google's models.
    
    Args:
        prompt: The input prompt
        model_name: The model to use (defaults to system default)
        stream: Whether to stream the response
        **kwargs: Additional parameters to pass to the model
        
    Returns:
        Generated text or a generator for streaming
    """
    if not model_name:
        model_name = default_model
    
    if not GOOGLE_AI_AVAILABLE:
        # Fallback mode - return a message
        if stream:
            def fallback_generator():
                message = f"[FALLBACK MODE] Would use {model_name} to respond to: {prompt[:100]}..."
                for char in message:
                    yield char
                    time.sleep(0.01)
            return fallback_generator()
        else:
            return f"[FALLBACK MODE] Would use {model_name} to respond to: {prompt[:100]}..."
    
    try:
        # Log the request
        request_id = int(time.time() * 1000)
        log_data = {
            "request_id": request_id,
            "model": model_name,
            "prompt_length": len(prompt),
            "streaming": stream
        }
        
        # Write to log file in CONFIG_DIR
        log_file = Path(box1_exports["LOGS_DIR"]) / "model_requests.jsonl"
        with open(log_file, "a") as f:
            f.write(json.dumps(log_data) + "\n")
        
        # Generate text
        if stream:
            return ai.generate_text(prompt, model_name=model_name, stream=True, **kwargs)
        else:
            return ai.generate_text(prompt, model_name=model_name, **kwargs)
    except Exception as e:
        error_msg = f"Error generating text: {str(e)}"
        print(f"❌ {error_msg}")
        if stream:
            def error_generator():
                for char in error_msg:
                    yield char
                    time.sleep(0.01)
            return error_generator()
        else:
            return error_msg

# Memory system for conversation history
class MemorySystem:
    def __init__(self, max_history: int = 10):
        self.max_history = max_history
        self.conversations: Dict[str, List[Dict[str, str]]] = {}
        self.current_conversation = "default"
        self.conversations[self.current_conversation] = []
    
    def add_message(self, role: str, content: str, conversation_id: str = None) -> None:
        """Add a message to the conversation history."""
        conv_id = conversation_id or self.current_conversation
        
        if conv_id not in self.conversations:
            self.conversations[conv_id] = []
        
        self.conversations[conv_id].append({
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        })
        
        # Trim history if needed
        if len(self.conversations[conv_id]) > self.max_history:
            self.conversations[conv_id] = self.conversations[conv_id][-self.max_history:]
    
    def get_conversation(self, conversation_id: str = None) -> List[Dict[str, str]]:
        """Get the conversation history."""
        conv_id = conversation_id or self.current_conversation
        return self.conversations.get(conv_id, [])
    
    def clear_conversation(self, conversation_id: str = None) -> None:
        """Clear the conversation history."""
        conv_id = conversation_id or self.current_conversation
        if conv_id in self.conversations:
            self.conversations[conv_id] = []
    
    def set_current_conversation(self, conversation_id: str) -> None:
        """Set the current conversation."""
        self.current_conversation = conversation_id
        if conversation_id not in self.conversations:
            self.conversations[conversation_id] = []
    
    def get_all_conversations(self) -> Dict[str, List[Dict[str, str]]]:
        """Get all conversations."""
        return self.conversations
    
    def save_to_file(self, file_path: str = None) -> str:
        """Save conversations to a file."""
        if not file_path:
            file_path = Path(box1_exports["CONFIG_DIR"]) / "conversations.json"
        
        with open(file_path, "w") as f:
            json.dump(self.conversations, f, indent=2)
        
        return file_path
    
    def load_from_file(self, file_path: str = None) -> bool:
        """Load conversations from a file."""
        if not file_path:
            file_path = Path(box1_exports["CONFIG_DIR"]) / "conversations.json"
        
        if not os.path.exists(file_path):
            return False
        
        try:
            with open(file_path, "r") as f:
                self.conversations = json.load(f)
            return True
        except Exception as e:
            print(f"❌ Error loading conversations: {e}")
            return False

# Initialize memory system
memory = MemorySystem()
memory_file = Path(box1_exports["CONFIG_DIR"]) / "conversations.json"
if memory_file.exists():
    memory.load_from_file(str(memory_file))
    print(f"✅ Loaded conversation history from {memory_file}")
else:
    print("ℹ️ No existing conversation history found")

# Chat function that uses memory
async def chat(message: str, conversation_id: str = None, model_name: str = None, stream: bool = False) -> Union[str, Generator]:
    """Chat with the model using conversation history."""
    conv_id = conversation_id or memory.current_conversation
    memory.add_message("user", message, conv_id)
    
    # Build prompt with conversation history
    conversation = memory.get_conversation(conv_id)
    prompt = "\n".join([f"{msg['role']}: {msg['content']}" for msg in conversation])
    
    # Generate response
    if stream:
        response_stream = generate_text(prompt, model_name, stream=True)
        response_chunks = []
        
        # Define an async generator to handle streaming
        async def stream_response():
            for chunk in response_stream:
                response_chunks.append(chunk)
                yield chunk
        
        # Return the generator
        return stream_response()
    else:
        response = generate_text(prompt, model_name)
        memory.add_message("assistant", response, conv_id)
        return response

# Test the model
if GOOGLE_AI_AVAILABLE:
    print("\n🧪 Testing model with a simple query...")
    response = generate_text("Hello, what can you do to help me with coding?")
    print(f"\nModel response:\n{response}")
else:
    print("\n⚠️ Skipping model test as Google AI is not available")

# Export configuration for other boxes
box2_exports = {
    "default_model": default_model,
    "available_models": available_models,
    "google_ai_available": GOOGLE_AI_AVAILABLE,
    "memory_initialized": True
}

box2_exports_file = CONFIG_DIR / "box2_exports.json"
with open(box2_exports_file, "w") as f:
    json.dump(box2_exports, f, indent=2)

print("\n🎉 BOX 2 SETUP COMPLETED SUCCESSFULLY!")
print(f"🧠 Default Model: {default_model}")
print(f"🔢 Available Models: {len(available_models)}")
print(f"💾 Memory System: Initialized")

# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🛠️ BOX 3: Tool Registry and Execution Framework - v1.0                                                   ║
# ║                                                                                                          ║
# ╟────────────────────────────────────── CORE FEATURES ─────────────────────────────────────────────────────╢
# ║ - Tool registration system                                                                               ║
# ║ - Tool execution pipeline                                                                                ║
# ║ - File system operations                                                                                 ║
# ║ - Code execution tools                                                                                   ║
# ║ - Research and deployment tools                                                                          ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

In [ ]:
print("🛠️ BOX 3: Initializing Tool Registry and Execution Framework...")

import os
import sys
import json
import time
import asyncio
import subprocess
import tempfile
import inspect
from pathlib import Path
from typing import Dict, List, Any, Optional, Union, Callable
from pydantic import BaseModel

# Load configuration from previous boxes
try:
    with open(Path("./GoogleManusSystem/config/box1_exports.json"), "r") as f:
        box1_exports = json.load(f)
    
    with open(Path("./GoogleManusSystem/config/box2_exports.json"), "r") as f:
        box2_exports = json.load(f)
    
    # Extract paths and configuration
    BASE_DIR = Path(box1_exports["BASE_DIR"])
    WORKSPACE_DIR = Path(box1_exports["WORKSPACE_DIR"])
    CONFIG_DIR = Path(box1_exports["CONFIG_DIR"])
    TOOLS_DIR = Path(box1_exports["TOOLS_DIR"])
    IS_COLAB = box1_exports["IS_COLAB"]
    
    # Get model information
    default_model = box2_exports["default_model"]
    GOOGLE_AI_AVAILABLE = box2_exports["google_ai_available"]
    
    print("✅ Loaded configuration from previous boxes")
except Exception as e:
    print(f"❌ Error loading previous box configurations: {e}")
    print("⚠️ Using default configuration")
    
    # Default configuration if previous boxes haven't been run
    BASE_DIR = Path("./GoogleManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    CONFIG_DIR = BASE_DIR / "config"
    TOOLS_DIR = BASE_DIR / "tools"
    IS_COLAB = True
    default_model = "google/gemini-2.5-pro"
    GOOGLE_AI_AVAILABLE = False

# Ensure workspace directory exists
WORKSPACE_DIR.mkdir(parents=True, exist_ok=True)
os.chdir(WORKSPACE_DIR)
print(f"📁 Working directory set to: {WORKSPACE_DIR}")

# Tool registry
TOOL_REGISTRY: Dict[str, Callable] = {}

# Tool registration decorator
def register_tool(name: str):
    """Decorator to register a tool in the registry."""
    def decorator(func):
        TOOL_REGISTRY[name] = func
        return func
    return decorator

# Tool call model
class ToolCall(BaseModel):
    tool_name: str
    tool_input: Dict[str, Any] = {}

# Task request model
class TaskRequest(BaseModel):
    task: str
    context: Optional[str] = None

# Tool execution function
async def call_tool(tool_call: ToolCall):
    """Execute a tool with the given input."""
    tool_name = tool_call.tool_name
    tool_input = tool_call.tool_input
    
    if tool_name not in TOOL_REGISTRY:
        return {"error": f"Tool {tool_name} not found"}
    
    try:
        result = TOOL_REGISTRY[tool_name](**tool_input)
        return result
    except Exception as e:
        return {"error": f"Error executing tool {tool_name}: {str(e)}"}

# List available tools
async def list_tools():
    """List all available tools with their descriptions and parameters."""
    tools_info = []
    
    for name, func in TOOL_REGISTRY.items():
        # Get function signature and docstring
        sig = inspect.signature(func)
        doc = inspect.getdoc(func) or ""
        
        # Extract parameters
        params = []
        for param_name, param in sig.parameters.items():
            param_info = {
                "name": param_name,
                "required": param.default == inspect.Parameter.empty,
                "type": str(param.annotation) if param.annotation != inspect.Parameter.empty else "Any"
            }
            params.append(param_info)
        
        tools_info.append({
            "name": name,
            "description": doc.split("\n")[0] if doc else "",
            "parameters": params
        })
    
    return {"tools": tools_info}

# ===== FILE SYSTEM TOOLS =====

@register_tool("write_file")
def write_file(file_path: str, content: str):
    """Write content to a file."""
    try:
        # Ensure the file path is within the workspace
        full_path = WORKSPACE_DIR / file_path
        
        # Create parent directories if they don't exist
        full_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Write the content
        with open(full_path, "w") as f:
            f.write(content)
        
        return {"success": True, "message": f"File written to {file_path}", "path": str(full_path)}
    except Exception as e:
        return {"success": False, "error": str(e)}

@register_tool("read_file")
def read_file(file_path: str):
    """Read content from a file."""
    try:
        # Ensure the file path is within the workspace
        full_path = WORKSPACE_DIR / file_path
        
        # Check if the file exists
        if not full_path.exists():
            return {"success": False, "error": f"File {file_path} not found"}
        
        # Read the content
        with open(full_path, "r") as f:
            content = f.read()
        
        return {"success": True, "content": content, "path": str(full_path)}
    except Exception as e:
        return {"success": False, "error": str(e)}

@register_tool("list_files")
def list_files(directory: str = ".", pattern: str = "*"):
    """List files in a directory."""
    try:
        # Ensure the directory path is within the workspace
        full_path = WORKSPACE_DIR / directory
        
        # Check if the directory exists
        if not full_path.exists():
            return {"success": False, "error": f"Directory {directory} not found"}
        
        # List files
        files = list(full_path.glob(pattern))
        file_list = [{
            "name": f.name,
            "path": str(f.relative_to(WORKSPACE_DIR)),
            "type": "directory" if f.is_dir() else "file",
            "size": f.stat().st_size if f.is_file() else None
        } for f in files]
        
        return {"success": True, "files": file_list, "directory": directory}
    except Exception as e:
        return {"success": False, "error": str(e)}

# ===== CODE EXECUTION TOOLS =====

@register_tool("execute_python")
def execute_python(code: str, timeout: int = 30):
    """Execute Python code and return the result."""
    try:
        # Create a temporary file
        with tempfile.NamedTemporaryFile(suffix=".py", delete=False) as temp_file:
            temp_file.write(code.encode())
            temp_path = temp_file.name
        
        # Execute the code
        result = subprocess.run(
            [sys.executable, temp_path],
            capture_output=True,
            text=True,
            timeout=timeout
        )
        
        # Clean up
        os.unlink(temp_path)
        
        return {
            "success": result.returncode == 0,
            "stdout": result.stdout,
            "stderr": result.stderr,
            "exit_code": result.returncode
        }
    except subprocess.TimeoutExpired:
        return {"success": False, "error": f"Execution timed out after {timeout} seconds"}
    except Exception as e:
        return {"success": False, "error": str(e)}

@register_tool("install_package")
def install_package(package_name: str):
    """Install a Python package."""
    try:
        # Execute pip install
        result = subprocess.run(
            [sys.executable, "-m", "pip", "install", package_name],
            capture_output=True,
            text=True
        )
        
        return {
            "success": result.returncode == 0,
            "stdout": result.stdout,
            "stderr": result.stderr,
            "exit_code": result.returncode
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# ===== RESEARCH TOOLS =====

@register_tool("search_web")
def search_web(query: str, num_results: int = 5):
    """Search the web for information (simulated in this version)."""
    # In a real implementation, this would use a search API
    # For now, we'll just return a message
    return {
        "success": True,
        "message": f"Would search for: {query} (limited to {num_results} results)",
        "results": []
    }

# ===== DEPLOYMENT TOOLS =====

@register_tool("launch_site")
def launch_site(site_path: str, port: int = 8000):
    """Launch a website locally."""
    try:
        # Ensure the site path is within the workspace
        full_path = WORKSPACE_DIR / site_path
        
        # Create the directory if it doesn't exist
        full_path.mkdir(parents=True, exist_ok=True)
        
        # Create a simple index.html if it doesn't exist
        index_path = full_path / "index.html"
        if not index_path.exists():
            with open(index_path, "w") as f:
                f.write(f"""<!DOCTYPE html>
                <html>
                <head>
                    <title>Google Manus Site</title>
                    <style>
                        body {{ font-family: Arial, sans-serif; margin: 40px; }}
                        h1 {{ color: #4285F4; }}
                    </style>
                </head>
                <body>
                    <h1>Welcome to Google Manus Site</h1>
                    <p>This is a simple website launched by Google Manus System.</p>
                    <p>Current time: {time.strftime('%Y-%m-%d %H:%M:%S')}</p>
                </body>
                </html>""")
        
        # Create a simple server script
        server_path = full_path / "server.py"
        with open(server_path, "w") as f:
            f.write(f"""
import http.server
import socketserver
import os

PORT = {port}
DIRECTORY = '{full_path}'

class Handler(http.server.SimpleHTTPRequestHandler):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, directory=DIRECTORY, **kwargs)
    
    def end_headers(self):
        self.send_header('Access-Control-Allow-Origin', '*')
        super().end_headers()

with socketserver.TCPServer(("", PORT), Handler) as httpd:
    print(f"Serving at port {{PORT}}")
    httpd.serve_forever()
            """)
        
        # Start the server in the background
        process = subprocess.Popen([sys.executable, str(server_path)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Wait a moment for the server to start
        time.sleep(2)
        
        # Check if the process is still running
        if process.poll() is not None:
            stdout, stderr = process.communicate()
            return {
                "success": False,
                "error": f"Server failed to start: {stderr.decode()}"
            }
        
        return {
            "success": True,
            "message": f"Site launched at http://localhost:{port}",
            "url": f"http://localhost:{port}",
            "site_path": str(full_path),
            "process_id": process.pid
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# ===== AGENT TOOLS =====

@register_tool("action_agent")
def action_agent(task: str, context: str = None):
    """Execute a task using the agent."""
    try:
        # In a real implementation, this would use the agent system
        # For now, we'll just use the model to generate a response
        
        # Import the generate_text function from Box 2
        from __main__ import generate_text
        
        prompt = f"Task: {task}\n"
        if context:
            prompt += f"Context: {context}\n"
        
        prompt += "\nPlease help me complete this task. Provide a step-by-step approach and any code or commands needed."
        
        response = generate_text(prompt)
        
        return {
            "success": True,
            "task": task,
            "response": response
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

@register_tool("get_agent_memory")
def get_agent_memory():
    """Get the agent's memory."""
    try:
        # Import the memory system from Box 2
        from __main__ import memory
        
        return {
            "success": True,
            "conversations": memory.get_all_conversations()
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

@register_tool("clear_agent_memory")
def clear_agent_memory(conversation_id: str = None):
    """Clear the agent's memory."""
    try:
        # Import the memory system from Box 2
        from __main__ import memory
        
        memory.clear_conversation(conversation_id)
        
        return {
            "success": True,
            "message": f"Memory cleared for conversation {conversation_id or 'default'}"
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# Print registered tools
print(f"✅ Registered {len(TOOL_REGISTRY)} tools")
for tool_name in TOOL_REGISTRY.keys():
    print(f"  - {tool_name}")

# Export configuration for other boxes
box3_exports = {
    "tools_registered": list(TOOL_REGISTRY.keys()),
    "workspace_dir": str(WORKSPACE_DIR)
}

box3_exports_file = CONFIG_DIR / "box3_exports.json"
with open(box3_exports_file, "w") as f:
    json.dump(box3_exports, f, indent=2)

print("\n🎉 BOX 3 SETUP COMPLETED SUCCESSFULLY!")
print(f"🛠️ Tools Registered: {len(TOOL_REGISTRY)}")
print(f"📁 Workspace Directory: {WORKSPACE_DIR}")

# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🖥️ BOX 4: User Interface and Interaction - v1.0                                                          ║
# ║                                                                                                          ║
# ╟────────────────────────────────────── CORE FEATURES ─────────────────────────────────────────────────────╢
# ║ - Jupyter widgets interface                                                                              ║
# ║ - Chat interface                                                                                         ║
# ║ - Task execution interface                                                                               ║
# ║ - Tool execution interface                                                                               ║
# ║ - Output display and formatting                                                                          ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

In [ ]:
print("🖥️ BOX 4: Initializing User Interface and Interaction...")

import os
import sys
import json
import time
import asyncio
from pathlib import Path
from typing import Dict, List, Any, Optional, Union, Callable
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Load configuration from previous boxes
try:
    with open(Path("./GoogleManusSystem/config/box1_exports.json"), "r") as f:
        box1_exports = json.load(f)
    
    with open(Path("./GoogleManusSystem/config/box2_exports.json"), "r") as f:
        box2_exports = json.load(f)
    
    with open(Path("./GoogleManusSystem/config/box3_exports.json"), "r") as f:
        box3_exports = json.load(f)
    
    # Extract configuration
    WORKSPACE_DIR = Path(box1_exports["WORKSPACE_DIR"])
    IS_COLAB = box1_exports["IS_COLAB"]
    default_model = box2_exports["default_model"]
    available_models = box2_exports["available_models"]
    GOOGLE_AI_AVAILABLE = box2_exports["google_ai_available"]
    tools_registered = box3_exports["tools_registered"]
    
    print("✅ Loaded configuration from previous boxes")
except Exception as e:
    print(f"❌ Error loading previous box configurations: {e}")
    print("⚠️ Using default configuration")
    
    # Default configuration if previous boxes haven't been run
    WORKSPACE_DIR = Path("./GoogleManusSystem/workspace")
    IS_COLAB = True
    default_model = "google/gemini-2.5-pro"
    available_models = ["google/gemini-2.5-pro"]
    GOOGLE_AI_AVAILABLE = False
    tools_registered = ["write_file", "read_file", "list_files", "execute_python", "action_agent"]

# Import functions from previous boxes
try:
    from __main__ import generate_text, chat, memory, call_tool, TOOL_REGISTRY
    print("✅ Imported functions from previous boxes")
except ImportError as e:
    print(f"❌ Error importing functions: {e}")
    print("⚠️ UI will have limited functionality")
    
    # Define placeholder functions
    def generate_text(prompt, model_name=None, stream=False):
        return f"[PLACEHOLDER] Would generate text for: {prompt[:50]}..."
    
    async def chat(message, conversation_id=None, model_name=None, stream=False):
        return f"[PLACEHOLDER] Would chat about: {message[:50]}..."
    
    class MemorySystem:
        def get_conversation(self, conversation_id=None):
            return []
        
        def add_message(self, role, content, conversation_id=None):
            pass
        
        def clear_conversation(self, conversation_id=None):
            pass
    
    memory = MemorySystem()
    
    async def call_tool(tool_call):
        return {"error": "Tool execution not available"}
    
    TOOL_REGISTRY = {}
    for tool in tools_registered:
        TOOL_REGISTRY[tool] = lambda **kwargs: {"error": "Tool not implemented"}

# ===== HELPER FUNCTIONS =====

def format_code(code, language="python"):
    """Format code with syntax highlighting."""
    return f"""<div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px; margin: 10px 0;">
    <pre><code class="{language}">{code}</code></pre>
    </div>"""

def format_message(role, content):
    """Format a chat message."""
    if role == "user":
        color = "#e6f7ff"
        icon = "👤"
    else:
        color = "#f0f0f0"
        icon = "🤖"
    
    return f"""<div style="background-color: {color}; padding: 10px; border-radius: 5px; margin: 10px 0;">
    <strong>{icon} {role.capitalize()}</strong>
    <div style="margin-top: 5px;">{content}</div>
    </div>"""

def format_tool_result(result):
    """Format a tool execution result."""
    if isinstance(result, dict) and "error" in result:
        return f"""<div style="background-color: #ffe6e6; padding: 10px; border-radius: 5px; margin: 10px 0;">
        <strong>❌ Error</strong>
        <div style="margin-top: 5px;">{result['error']}</div>
        </div>"""
    
    if isinstance(result, dict) and "success" in result and not result["success"]:
        return f"""<div style="background-color: #ffe6e6; padding: 10px; border-radius: 5px; margin: 10px 0;">
        <strong>❌ Error</strong>
        <div style="margin-top: 5px;">{result.get('error', 'Unknown error')}</div>
        </div>"""
    
    # Format success result
    result_html = "<div style=\"background-color: #e6ffe6; padding: 10px; border-radius: 5px; margin: 10px 0;\">\n"
    result_html += "<strong>✅ Success</strong>\n"
    result_html += "<div style=\"margin-top: 5px;\">\n"
    
    if isinstance(result, dict):
        for key, value in result.items():
            if key in ["success", "error"]:
                continue
            
            if key == "stdout" and value:
                result_html += f"<strong>Output:</strong>\n"
                result_html += f"<pre>{value}</pre>\n"
            elif key == "stderr" and value:
                result_html += f"<strong>Errors:</strong>\n"
                result_html += f"<pre>{value}</pre>\n"
            elif key == "content" and value:
                result_html += f"<strong>Content:</strong>\n"
                result_html += f"<pre>{value}</pre>\n"
            elif key == "files" and value:
                result_html += f"<strong>Files:</strong>\n"
                result_html += "<ul>\n"
                for file in value:
                    result_html += f"<li>{file['name']} ({file['type']})</li>\n"
                result_html += "</ul>\n"
            elif key == "message" and value:
                result_html += f"<strong>Message:</strong> {value}\n"
            elif key == "url" and value:
                result_html += f"<strong>URL:</strong> <a href=\"{value}\" target=\"_blank\">{value}</a>\n"
            elif key == "response" and value:
                result_html += f"<strong>Response:</strong>\n"
                result_html += f"<div>{value}</div>\n"
    else:
        result_html += f"<pre>{result}</pre>\n"
    
    result_html += "</div>\n</div>"
    
    return result_html

# ===== UI COMPONENTS =====

# Header
header = widgets.HTML(
    value="<h1>🤖 Google Manus System</h1>"
)

# Model selection
model_dropdown = widgets.Dropdown(
    options=available_models,
    value=default_model,
    description='Model:',
    disabled=not GOOGLE_AI_AVAILABLE,
    style={'description_width': '100px'}
)

# Tabs for different interfaces
tab = widgets.Tab()
tab_contents = []

# ===== CHAT INTERFACE =====

chat_output = widgets.Output()
chat_input = widgets.Text(
    value='',
    placeholder='Type a message...',
    description='',
    disabled=False
)
chat_button = widgets.Button(
    description='Send',
    disabled=False,
    button_style='primary',
    tooltip='Send message',
    icon='paper-plane'
)
clear_chat_button = widgets.Button(
    description='Clear Chat',
    disabled=False,
    button_style='danger',
    tooltip='Clear chat history',
    icon='trash'
)

chat_controls = widgets.HBox([chat_input, chat_button, clear_chat_button])
chat_interface = widgets.VBox([chat_output, chat_controls])

# Chat event handlers
def on_chat_button_clicked(b):
    message = chat_input.value
    if not message.strip():
        return
    
    chat_input.value = ''
    
    with chat_output:
        display(HTML(format_message("user", message)))
        
        # Add to memory
        memory.add_message("user", message)
        
        # Generate response
        response = generate_text(message, model_dropdown.value)
        
        # Add to memory
        memory.add_message("assistant", response)
        
        # Display response
        display(HTML(format_message("assistant", response)))

def on_chat_input_submitted(sender):
    on_chat_button_clicked(None)

def on_clear_chat_button_clicked(b):
    memory.clear_conversation()
    with chat_output:
        clear_output()
        display(HTML("<p>Chat history cleared.</p>"))

chat_button.on_click(on_chat_button_clicked)
chat_input.on_submit(on_chat_input_submitted)
clear_chat_button.on_click(on_clear_chat_button_clicked)

# ===== TASK EXECUTION INTERFACE =====

task_input = widgets.Text(
    value='',
    placeholder='Enter a task...',
    description='Task:',
    disabled=False,
    style={'description_width': '100px'}
)
task_context = widgets.Textarea(
    value='',
    placeholder='Optional context...',
    description='Context:',
    disabled=False,
    style={'description_width': '100px'}
)
task_button = widgets.Button(
    description='Execute Task',
    disabled=False,
    button_style='success',
    tooltip='Execute the task',
    icon='play'
)
task_output = widgets.Output()

task_interface = widgets.VBox([task_input, task_context, task_button, task_output])

# Task event handlers
def on_task_button_clicked(b):
    task = task_input.value
    context = task_context.value
    
    if not task.strip():
        return
    
    with task_output:
        clear_output()
        display(HTML(f"<p><strong>Task:</strong> {task}</p>"))
        if context:
            display(HTML(f"<p><strong>Context:</strong> {context}</p>"))
        
        display(HTML("<p><em>Executing task...</em></p>"))
        
        # Call the action_agent tool
        if "action_agent" in TOOL_REGISTRY:
            result = TOOL_REGISTRY["action_agent"](task=task, context=context)
            display(HTML(format_tool_result(result)))
        else:
            display(HTML("<p>❌ Action agent tool not available.</p>"))

task_button.on_click(on_task_button_clicked)

# ===== TOOL EXECUTION INTERFACE =====

tool_dropdown = widgets.Dropdown(
    options=tools_registered,
    value=tools_registered[0] if tools_registered else None,
    description='Tool:',
    disabled=not tools_registered,
    style={'description_width': '100px'}
)
tool_input = widgets.Textarea(
    value='{}',
    placeholder='Enter tool input as JSON...',
    description='Input:',
    disabled=False,
    style={'description_width': '100px'}
)
tool_button = widgets.Button(
    description='Execute Tool',
    disabled=False,
    button_style='info',
    tooltip='Execute the selected tool',
    icon='wrench'
)
tool_output = widgets.Output()

tool_interface = widgets.VBox([tool_dropdown, tool_input, tool_button, tool_output])

# Tool event handlers
def on_tool_dropdown_change(change):
    tool_name = change['new']
    if tool_name == "write_file":
        tool_input.value = '{"file_path": "example.txt", "content": "Hello, world!"}'
    elif tool_name == "read_file":
        tool_input.value = '{"file_path": "example.txt"}'
    elif tool_name == "list_files":
        tool_input.value = '{"directory": ".", "pattern": "*"}'
    elif tool_name == "execute_python":
        tool_input.value = '{"code": "print(\\'Hello, world!\\')\\nfor i in range(5):\\n    print(i)"}'
    elif tool_name == "action_agent":
        tool_input.value = '{"task": "Write a Python function to calculate Fibonacci numbers", "context": "The function should be efficient"}'
    elif tool_name == "launch_site":
        tool_input.value = '{"site_path": "my_site", "port": 8000}'
    else:
        tool_input.value = '{}'

def on_tool_button_clicked(b):
    tool_name = tool_dropdown.value
    input_json = tool_input.value
    
    try:
        input_dict = json.loads(input_json)
    except json.JSONDecodeError as e:
        with tool_output:
            clear_output()
            display(HTML(f"<p>❌ Invalid JSON: {str(e)}</p>"))
        return
    
    with tool_output:
        clear_output()
        display(HTML(f"<p><strong>Executing tool:</strong> {tool_name}</p>"))
        display(HTML(f"<p><strong>Input:</strong> {input_json}</p>"))
        
        # Execute the tool
        if tool_name in TOOL_REGISTRY:
            try:
                result = TOOL_REGISTRY[tool_name](**input_dict)
                display(HTML(format_tool_result(result)))
            except Exception as e:
                display(HTML(f"<p>❌ Error executing tool: {str(e)}</p>"))
        else:
            display(HTML("<p>❌ Tool not found.</p>"))

tool_dropdown.observe(on_tool_dropdown_change, names='value')
tool_button.on_click(on_tool_button_clicked)

# ===== CODE EDITOR INTERFACE =====

code_editor = widgets.Textarea(
    value='# Enter your Python code here\nprint("Hello, world!")',
    placeholder='Enter Python code...',
    description='Code:',
    disabled=False,
    style={'description_width': '100px'}
)
code_run_button = widgets.Button(
    description='Run Code',
    disabled=False,
    button_style='success',
    tooltip='Execute the code',
    icon='play'
)
code_output = widgets.Output()

code_interface = widgets.VBox([code_editor, code_run_button, code_output])

# Code editor event handlers
def on_code_run_button_clicked(b):
    code = code_editor.value
    
    with code_output:
        clear_output()
        display(HTML("<p><strong>Executing code...</strong></p>"))
        
        # Execute the code
        if "execute_python" in TOOL_REGISTRY:
            result = TOOL_REGISTRY["execute_python"](code=code)
            display(HTML(format_tool_result(result)))
        else:
            display(HTML("<p>❌ Code execution tool not available.</p>"))

code_run_button.on_click(on_code_run_button_clicked)

# ===== ASSEMBLE THE UI =====

# Add tabs
tab_contents = [chat_interface, task_interface, tool_interface, code_interface]
tab.children = tab_contents
tab.set_title(0, "Chat")
tab.set_title(1, "Task Execution")
tab.set_title(2, "Tool Execution")
tab.set_title(3, "Code Editor")

# Main UI
main_ui = widgets.VBox([header, model_dropdown, tab])

# Display the UI
display(main_ui)

# Initialize the chat interface
with chat_output:
    display(HTML("<p>Welcome to Google Manus System! How can I help you today?</p>"))

print("\n🎉 BOX 4 SETUP COMPLETED SUCCESSFULLY!")
print("🖥️ User Interface Initialized")
print("💬 Chat Interface Ready")
print("🎯 Task Execution Ready")
print("🛠️ Tool Execution Ready")
print("💻 Code Editor Ready")

# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🌐 BOX 5: Web Deployment and External Services - v1.0                                                     ║
# ║                                                                                                          ║
# ╟────────────────────────────────────── CORE FEATURES ─────────────────────────────────────────────────────╢
# ║ - FastAPI server setup                                                                                   ║
# ║ - FRP tunnel integration                                                                                 ║
# ║ - Static file serving                                                                                    ║
# ║ - Web-based code execution environment                                                                   ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

In [ ]:
print("🌐 BOX 5: Initializing Web Deployment and External Services...")

import os
import sys
import json
import time
import asyncio
import threading
import subprocess
from pathlib import Path
from typing import Dict, List, Any, Optional, Union, Callable

# Load configuration from previous boxes
try:
    with open(Path("./GoogleManusSystem/config/box1_exports.json"), "r") as f:
        box1_exports = json.load(f)
    
    with open(Path("./GoogleManusSystem/config/box2_exports.json"), "r") as f:
        box2_exports = json.load(f)
    
    with open(Path("./GoogleManusSystem/config/box3_exports.json"), "r") as f:
        box3_exports = json.load(f)
    
    # Extract configuration
    BASE_DIR = Path(box1_exports["BASE_DIR"])
    WORKSPACE_DIR = Path(box1_exports["WORKSPACE_DIR"])
    SITES_DIR = Path(box1_exports["SITES_DIR"])
    IS_COLAB = box1_exports["IS_COLAB"]
    default_model = box2_exports["default_model"]
    GOOGLE_AI_AVAILABLE = box2_exports["google_ai_available"]
    tools_registered = box3_exports["tools_registered"]
    
    print("✅ Loaded configuration from previous boxes")
except Exception as e:
    print(f"❌ Error loading previous box configurations: {e}")
    print("⚠️ Using default configuration")
    
    # Default configuration if previous boxes haven't been run
    BASE_DIR = Path("./GoogleManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    SITES_DIR = BASE_DIR / "sites"
    IS_COLAB = True
    default_model = "google/gemini-2.5-pro"
    GOOGLE_AI_AVAILABLE = False
    tools_registered = ["write_file", "read_file", "list_files", "execute_python", "action_agent"]

# Import functions from previous boxes
try:
    from __main__ import generate_text, chat, memory, call_tool, TOOL_REGISTRY
    print("✅ Imported functions from previous boxes")
except ImportError as e:
    print(f"❌ Error importing functions: {e}")
    print("⚠️ Web server will have limited functionality")
    
    # Define placeholder functions
    def generate_text(prompt, model_name=None, stream=False):
        return f"[PLACEHOLDER] Would generate text for: {prompt[:50]}..."
    
    async def chat(message, conversation_id=None, model_name=None, stream=False):
        return f"[PLACEHOLDER] Would chat about: {message[:50]}..."
    
    class MemorySystem:
        def get_conversation(self, conversation_id=None):
            return []
        
        def add_message(self, role, content, conversation_id=None):
            pass
        
        def clear_conversation(self, conversation_id=None):
            pass
    
    memory = MemorySystem()
    
    async def call_tool(tool_call):
        return {"error": "Tool execution not available"}
    
    TOOL_REGISTRY = {}
    for tool in tools_registered:
        TOOL_REGISTRY[tool] = lambda **kwargs: {"error": "Tool not implemented"}

# Create FastAPI server
from fastapi import FastAPI, Request, BackgroundTasks, HTTPException, Depends
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import JSONResponse, HTMLResponse, FileResponse
from pydantic import BaseModel
import uvicorn

app = FastAPI(title="Google Manus API", description="API for Google Manus System", version="1.0.0")

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# API models
class ChatRequest(BaseModel):
    message: str
    conversation_id: Optional[str] = None
    model_name: Optional[str] = None

class ToolRequest(BaseModel):
    tool_name: str
    tool_input: Dict[str, Any] = {}

class TaskRequest(BaseModel):
    task: str
    context: Optional[str] = None
    model_name: Optional[str] = None

# API routes
@app.get("/", response_class=HTMLResponse)
async def root():
    return """
    <html>
        <head>
            <title>Google Manus API</title>
            <style>
                body { font-family: Arial, sans-serif; margin: 40px; line-height: 1.6; }
                h1 { color: #4285F4; }
                h2 { color: #34A853; margin-top: 30px; }
                pre { background-color: #f5f5f5; padding: 10px; border-radius: 5px; }
                .endpoint { margin-bottom: 20px; }
            </style>
        </head>
        <body>
            <h1>🤖 Google Manus API</h1>
            <p>Welcome to the Google Manus API. Use the following endpoints to interact with the system:</p>
            
            <div class="endpoint">
                <h2>📋 API Documentation</h2>
                <p>View the full API documentation at <a href="/docs">/docs</a></p>
            </div>
            
            <div class="endpoint">
                <h2>💬 Chat</h2>
                <p>Send a message to the model:</p>
                <pre>POST /api/chat
{
    "message": "Hello, how are you?",
    "conversation_id": "optional-id",
    "model_name": "optional-model-name"
}</pre>
            </div>
            
            <div class="endpoint">
                <h2>🛠️ Tool Execution</h2>
                <p>Execute a tool:</p>
                <pre>POST /api/tool
{
    "tool_name": "write_file",
    "tool_input": {
        "file_path": "example.txt",
        "content": "Hello, world!"
    }
}</pre>
            </div>
            
            <div class="endpoint">
                <h2>🎯 Task Execution</h2>
                <p>Execute a task:</p>
                <pre>POST /api/task
{
    "task": "Write a Python function to calculate Fibonacci numbers",
    "context": "The function should be efficient",
    "model_name": "optional-model-name"
}</pre>
            </div>
            
            <div class="endpoint">
                <h2>🧠 Models</h2>
                <p>List available models:</p>
                <pre>GET /api/models</pre>
            </div>
            
            <div class="endpoint">
                <h2>🛠️ Tools</h2>
                <p>List available tools:</p>
                <pre>GET /api/tools</pre>
            </div>
        </body>
    </html>
    """

@app.get("/api/models")
async def get_models():
    try:
        from __main__ import available_models
        return {"models": available_models, "default_model": default_model}
    except ImportError:
        return {"models": [default_model], "default_model": default_model}

@app.get("/api/tools")
async def get_tools():
    return {"tools": tools_registered}

@app.post("/api/chat")
async def api_chat(request: ChatRequest):
    try:
        response = await chat(
            message=request.message,
            conversation_id=request.conversation_id,
            model_name=request.model_name
        )
        return {"response": response}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/api/tool")
async def api_tool(request: ToolRequest):
    try:
        if request.tool_name not in TOOL_REGISTRY:
            raise HTTPException(status_code=404, detail=f"Tool {request.tool_name} not found")
        
        result = TOOL_REGISTRY[request.tool_name](**request.tool_input)
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/api/task")
async def api_task(request: TaskRequest):
    try:
        if "action_agent" not in TOOL_REGISTRY:
            raise HTTPException(status_code=404, detail="Action agent tool not found")
        
        result = TOOL_REGISTRY["action_agent"](task=request.task, context=request.context)
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Mount static files for sites
app.mount("/site", StaticFiles(directory=str(SITES_DIR)), name="sites")

# Create a simple site launcher
@app.get("/site/launch/{site_name}")
async def launch_site(site_name: str):
    site_path = SITES_DIR / site_name
    
    if not site_path.exists():
        site_path.mkdir(parents=True, exist_ok=True)
        
        # Create a simple index.html
        index_path = site_path / "index.html"
        with open(index_path, "w") as f:
            f.write(f"""
            <!DOCTYPE html>
            <html>
            <head>
                <title>{site_name} - Google Manus Site</title>
                <style>
                    body {{ font-family: Arial, sans-serif; margin: 40px; }}
                    h1 {{ color: #4285F4; }}
                </style>
            </head>
            <body>
                <h1>Welcome to {site_name}</h1>
                <p>This is a simple website launched by Google Manus System.</p>
                <p>Current time: {time.strftime('%Y-%m-%d %H:%M:%S')}</p>
            </body>
            </html>
            """)
    
    # Return the index.html file
    return FileResponse(site_path / "index.html")

# Setup FRP tunnel
def setup_frp(port: int = 8000):
    """Setup FRP tunnel for public access."""
    try:
        # Download FRP if not exists
        frp_dir = BASE_DIR / "frp"
        if not frp_dir.exists():
            print("⏳ Downloading FRP...")
            os.makedirs(frp_dir, exist_ok=True)
            subprocess.run(["wget", "https://github.com/fatedier/frp/releases/download/v0.51.3/frp_0.51.3_linux_amd64.tar.gz", "-O", str(frp_dir / "frp.tar.gz")])
            subprocess.run(["tar", "-xzf", str(frp_dir / "frp.tar.gz"), "-C", str(frp_dir), "--strip-components=1"])
            os.remove(str(frp_dir / "frp.tar.gz"))
            print("✅ FRP downloaded and extracted")
        
        # Create FRP config
        frp_config = f"""
[common]
server_addr = frp.example.com
server_port = 7000
token = your_token_here

[web]
type = http
local_port = {port}
custom_domains = your-domain.example.com
        """
        
        with open(frp_dir / "frpc.ini", "w") as f:
            f.write(frp_config)
        
        print("⚠️ FRP configuration created, but not started")
        print("ℹ️ To use FRP, you need to configure a real FRP server")
        print(f"ℹ️ Edit the configuration file at {frp_dir / 'frpc.ini'}")
        print(f"ℹ️ Then run: {frp_dir / 'frpc'} -c {frp_dir / 'frpc.ini'}")
        
        return {"success": True, "message": "FRP configuration created"}
    except Exception as e:
        print(f"❌ Error setting up FRP: {e}")
        return {"success": False, "error": str(e)}

# Start the server in a separate thread
def start_server():
    """Start the FastAPI server."""
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Setup FRP
frp_result = setup_frp()

# Start the server in a background thread
server_thread = threading.Thread(target=start_server)
server_thread.daemon = True  # Allow the thread to be terminated when the notebook is closed
server_thread.start()

print("\n🎉 BOX 5 SETUP COMPLETED SUCCESSFULLY!")
print("🌐 Web Server Started at http://localhost:8000")
print("📚 API Documentation available at http://localhost:8000/docs")
print("🚀 Site Launcher available at http://localhost:8000/site/launch/{site_name}")
print("⚙️ FRP Configuration Created (Manual Setup Required)")